In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import zarr
import dask.array as da
import dask
from dask.diagnostics import ProgressBar
from dask.array import stats

import torch
from torch.utils.data import Dataset
import pytorch_lightning as pl
import json
from tqdm import tqdm

In [3]:
from sdofm.datasets.ZarrIrradianceDataModuleHMI import ZarrIrradianceDataModuleHMI

In [12]:
hmi_path = "/mnt/sdoml/HMI.zarr"
aia_path = "/mnt/sdoml/AIA.zarr"
eve_path = "/mnt/sdoml/EVE_legacy.zarr"
components = ["Bx", "By", "Bz"]
wavelengths = ["131A","1600A","1700A","171A","193A","211A","304A","335A","94A"]
ions = ["C III", "Fe IX", "Fe VIII", "Fe X", "Fe XI", "Fe XII", "Fe XIII", "Fe XIV", "Fe XIX", "Fe XV", "Fe XVI", "Fe XVIII", "Fe XVI_2", "Fe XX", "Fe XX_2", "Fe XX_3", "H I", "H I_2", "H I_3", "He I", "He II", "He II_2", "He I_2", "Mg IX", "Mg X", "Mg X_2", "Ne VII", "Ne VIII", "O II", "O III", "O III_2", "O II_2", "O IV", "O IV_2", "O V", "O VI", "S XIV", "Si XII", "Si XII_2"]
frequency = "12min"
batch_size = 32

test = ZarrIrradianceDataModuleHMI(hmi_path, aia_path, eve_path, components, wavelengths, ions, frequency, batch_size, num_workers=16, val_months=[10,1], test_months=[11,12],  holdout_months=[], cache_dir="/home/walsh/repos/SDO-FM/notebooks/")


Data alignment calculation begin:
--------------------------------------------------
Aligning AIA data
Aligning AIA data for wavelength: 131A


100%|██████████| 5/5 [00:24<00:00,  4.98s/it]


Aligning AIA data for wavelength: 1600A


100%|██████████| 5/5 [00:23<00:00,  4.65s/it]


Aligning AIA data for wavelength: 1700A


...

Aligning AIA data for wavelength: 94A


100%|██████████| 5/5 [00:20<00:00,  4.18s/it]


AIA alignment completed with 177481 samples.
Aligning HMI data
Aligning HMI data for component: Bx


100%|██████████| 5/5 [00:16<00:00,  3.27s/it]


HMI alignment completed with 164426 samples.
Aligning EVE data

##################################################
[*] Total Alignment Completed with 16 Samples.
##################################################


Calculating normalizations for wavelength 131A:
--------------------------------------------------
Sum:
[########################################] | 100% Completed | 2.32 ss
Max Pixel Value:
[########################################] | 100% Completed | 202.89 ms
Standard Deviation:
[########################################] | 100% Completed | 203.06 ms
Skew:
[########################################] | 100% Completed | 404.22 ms
Kurtosis:
[########################################] | 100% Completed | 404.27 ms

Calculating normalizations for wavelength 1600A:
--------------------------------------------------
Sum:
[########################################] | 100% Completed | 1.92 ss
Max Pixel Value:
[########################################] | 100% Completed | 304.39 ms
Stand

In [8]:
hmi_data = zarr.group(zarr.DirectoryStore("/mnt/sdoml/EVE_legacy.zarr/"))

In [9]:
hmi_data.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='C III (2137380,) floa…

In [11]:
a = ["C III", "Fe IX", "Fe VIII", "Fe X", "Fe XI", "Fe XII", "Fe XIII", "Fe XIV", "Fe XIX", "Fe XV", "Fe XVI", "Fe XVIII", "Fe XVI_2", "Fe XX", "Fe XX_2", "Fe XX_3", "H I", "H I_2", "H I_3", "He I", "He II", "He II_2", "He I_2", "Mg IX", "Mg X", "Mg X_2", "Ne VII", "Ne VIII", "O II", "O III", "O III_2", "O II_2", "O IV", "O IV_2", "O V", "O VI", "S XIV", "Si XII", "Si XII_2"]
len(a)

39